In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import pandas as pd

In [2]:
df=pd.read_csv("C:\\Users\\shubh\\Desktop\\LnB_ML&Ai\\flipkart.csv")
df.head(2)

,Unnamed: 0,Product_name,Review,Rating
0,0,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Best under 60k Great performanceI got it for a...,5
1,1,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Good perfomence...,5


In [3]:
def create_labels(rating):
    if rating>=4:
        return 2 #postive
    elif rating==3:
        return 1 #neutral
    else:
        return 0 #negative

In [4]:
df['labels']=df['Rating'].apply(create_labels)

In [5]:
df

,Unnamed: 0,Product_name,Review,Rating,labels
0,0,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Best under 60k Great performanceI got it for a...,5,2
1,1,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Good perfomence...,5,2
2,2,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Great performance but usually it has also that...,5,2
3,3,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,My wife is so happy and best product 👌🏻😘,5,2
4,4,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,"Light weight laptop with new amazing features,...",5,2
...,...,...,...,...,...
2299,2299,MSI 27 inch Full HD IPS Panel Monitor (PRO MP2...,"Great display, accurate colours at this price ...",5,2
2300,2300,MSI 27 inch Full HD IPS Panel Monitor (PRO MP2...,Superb monitor first brought 1 used for 2 mont...,5,2
2301,2301,MSI 27 inch Full HD IPS Panel Monitor (PRO MP2...,Awesome,5,2
2302,2302,MSI 27 inch Full HD IPS Panel Monitor (PRO MP2...,Only one issue with adapter,5,2


In [6]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['Review'])
sequences=tokenizer.texts_to_sequences(df['Review'])
data=pad_sequences(sequences,maxlen=200)
data

array([[   0,    0,    0, ...,  101,  238, 2248],
       [   0,    0,    0, ...,    0,    3, 3127],
       [   0,    0,    0, ...,  127,  342,  312],
       ...,
       [   0,    0,    0, ...,    0,    0,   41],
       [   0,    0,    0, ...,  228,   17, 1493],
       [   0,    0,    0, ...,   46,  292,    6]])

In [7]:
x_train,x_test,y_train,y_test=train_test_split(data,df['labels'],test_size=0.2,random_state=42)


In [8]:
y_train=to_categorical(y_train,num_classes=3)
y_test=to_categorical(y_test,num_classes=3)

In [9]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(units=128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(units=3,activation='softmax'))

c:\Users\shubh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)
loss,accuracy=model.evaluate(x_test,y_test)
print(f"Loss is {loss} and accuracy is {accuracy}")


Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 10s 132ms/step - accuracy: 0.8151 - loss: 0.7715 - val_accuracy: 0.8266 - val_loss: 0.5720
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - accuracy: 0.8416 - loss: 0.4834 - val_accuracy: 0.8482 - val_loss: 0.4795
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 119ms/step - accuracy: 0.8858 - loss: 0.3241 - val_accuracy: 0.8753 - val_loss: 0.4022
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 111ms/step - accuracy: 0.9310 - loss: 0.2228 - val_accuracy: 0.8916 - val_loss: 0.3866
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 119ms/step - accuracy: 0.9633 - loss: 0.1352 - val_accuracy: 0.9079 - val_loss: 0.3458
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.9733 - loss: 0.1078 - val_accuracy: 0.9024 - val_loss: 0.3277
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.9784 - loss: 0.0740 - val_accuracy: 0.9133 - val_loss: 0.3548
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 146ms/step - accuracy: 0.9820 - loss: 0.0629 - val_accuracy: 0

In [15]:
def predict_user_input(text):
    sequences=tokenizer.texts_to_sequences([text])
    padded_sequence=pad_sequences(sequences,maxlen=200)
    prediction=model.predict(padded_sequence)
    label=prediction.argmax(axis=1)[0]
    if label==2:
        return "Customer Satisfied"
    elif label==1:
        return "Custemer partially Satisfied"
    else:
        return "Customer not satisfied"
user_review=input("Enter your review")
result=predict_user_input(user_review)

print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Customer not satisfied
